# DL4G - Jass Introduction

In this exercise we will look at some properties of the jass kit environment that can be used to develop your own jass agent.

You will need to have numpy installed, as well as the jass-kit environment.

In [4]:
from jass.game.game_util import *
from jass.game.game_sim import GameSim
from jass.game.game_observation import GameObservation
from jass.game.const import *
from jass.game.rule_schieber import RuleSchieber
from jass.agents.agent import Agent
from jass.agents.agent_random_schieber import AgentRandomSchieber
from jass.arena.arena import Arena

import json

import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
 # import seaborn as sns

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline

from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import KFold

from tqdm.notebook import tqdm
import ipywidgets as widgets
%matplotlib inline


Information about the cards is stored as one-hot encoded arrays, there are several tools available to access the information in the cards. 

Lets deal some random cards first.

In [5]:
# Lets set the seed of the random number generater, so that we get the same results
np.random.seed(1)

# This distributes the cards randomly among the 4 players.
hands = deal_random_hand()
print(hands.shape)
print(hands[0])

(4, 36)
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0]


In [6]:
# There is an entry for each player, to access the cards of the first player
cards = hands[0,:]
print(cards)

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0]


In [7]:
# This should be 9 cards
assert(cards.sum() == 9)

# The cards can be converted to other formats for easier reading or processing
print(convert_one_hot_encoded_cards_to_str_encoded_list(cards))

# Each card is encoded as a value between 0 and 35.
print(convert_one_hot_encoded_cards_to_int_encoded_list(cards))


['DJ', 'H6', 'SK', 'SJ', 'S9', 'CK', 'CQ', 'CJ', 'C7']
[3, 17, 19, 21, 23, 28, 29, 30, 34]


In [8]:
# There is a method to count colors too
colors = count_colors(cards)
print(colors)

[1 1 3 4]


There is a common jass "rule" to select trump, when you have the "Puur" (Jack of trump) and 3 or more other cards of the same color. 

Task 1: Write a function that returns an array of 4 values that contains a 1 for each color that fulfills the rule or 0 otherwise, i.e. [0 0 0 0] is returned, if you do not have any color with Jack and 3 other cards.


In [9]:
def havePuurWithFour(hand: np.ndarray) -> np.ndarray:
    encoded = convert_one_hot_encoded_cards_to_str_encoded_list(hand)
    result = np.zeros(4, dtype=int)
    colorIds = ['D','H','S','C']
    # D H S C
    # J ---> wenn jo denn farbe 4
    colors = count_colors(hand)
    for i, color in enumerate (colors):
        if color >=  4:
            if colorIds[i] + 'J' in encoded:
                result[i] = 1

    print(f"dhsc:{result}")
    return result
        

In [10]:
assert (havePuurWithFour(cards) == [0, 0, 0, 1]).all()
cards_2 = hands[1,:]
assert (havePuurWithFour(cards_2) == [0, 0, 0, 0]).all()

dhsc:[0 0 0 1]
dhsc:[0 0 0 0]


Another possibility to select trump is by assigning a value to each card, depending on whether the color is trump or not. This table is from the Maturawork of Daniel Graf from 2009: "Jassen auf Basis der Spieltheorie".

In [11]:
# Score for each card of a color from Ace to 6

# score if the color is trump
trump_score = [15, 10, 7, 25, 6, 19, 5, 5, 5]
# score if the color is not trump
no_trump_score = [9, 7, 5, 2, 1, 0, 0, 0, 0]
# score if obenabe is selected (all colors)
obenabe_score = [14, 10, 8, 7, 5, 0, 5, 0, 0,]
# score if uneufe is selected (all colors)
uneufe_score = [0, 2, 1, 1, 5, 5, 7, 9, 11]

Task 2: Implement a function that evaluates a hand that is given as a list of 9 cards and with a given trump value and returns a score depending on the table above. For example the score of our hand ['DJ', 'H6', 'SK', 'SJ', 'S9', 'CK', 'CQ', 'CJ', 'C7'] when Club is trump should be:

2 + 0 + 7 + 2 + 0 + 10 + 7 + 25 + 5 = 58

while the score is 70 if Spade is selected, which is better as you have both the jack and the nine.

You can use the arrays offset_of_card and color_of_card to get the offset (Ace, King, etc.) and color of a card.

In [12]:
def calculate_trump_selection_score(cards, trump: int) -> int:
    result = 0
    for card in cards:
        color = color_of_card[card]
        offset = offset_of_card[card]
        if color == trump:
            result += trump_score[offset]
        else:
            result += no_trump_score[offset]
    return result

In [13]:
card_list = convert_one_hot_encoded_cards_to_int_encoded_list(cards)
print(card_list)

[3, 17, 19, 21, 23, 28, 29, 30, 34]


In [111]:

card_list = convert_one_hot_encoded_cards_to_int_encoded_list(cards)
assert calculate_trump_selection_score(card_list, CLUBS) == 58
assert calculate_trump_selection_score(card_list, SPADES) == 70

## Agents

In order to play a game you have to program an agent that decides on the action. For that you have to override the methods action_trump and action_play_card.

Task 3: Use the function implemented above to select the best trump value. If the calculated trump value is below a threshold (for example let us take 68, as suggested in the work by Daniel Graf) you should "Schiebe", i.e. pass to your partner if you are still allowed to do that.

The game observation allows you to access the information about your card, and if you are the first or second player to select trump.

For playing a card, we just take a random action.

In [194]:
class MyAgent(Agent):
    hasStrongHand = False

    def __init__(self):
        super().__init__()
        # we need a rule object to determine the valid cards
        self._rule = RuleSchieber()
        
    def action_trump(self, obs: GameObservation) -> int:
        """
        Determine trump action for the given observation
        Args:
            obs: the game observation, it must be in a state for trump selection

        Returns:
            selected trump as encoded in jass.game.const or jass.game.const.PUSH
        """

        valid_cards = self._rule.get_valid_cards_from_obs(obs)

        # Calling DataFrame constructor after zipping
        # both lists, with columns specified
        cards = [
        # Diamonds
        'DA','DK','DQ','DJ','D10','D9','D8','D7','D6',
        # Hearts
        'HA','HK','HQ','HJ','H10','H9','H8','H7','H6',
        # Spades
        'SA','SK','SQ','SJ','S10','S9','S8','S7','S6',
        # Clubs
        'CA','CK','CQ','CJ','C10','C9','C8','C7','C6'
        ]
        forehand = ['FH']

        feature_columns = cards + forehand

        a = np.append(valid_cards,[obs.forehand+1])
    
        df = pd.DataFrame(a.reshape(-1, len(a)),columns=feature_columns)
        print (df)

        print(df)
        for color in 'DHSC':
        
            # Jack and nine combination
            new_col = '{}_J9'.format(color)
            df[new_col]  = df['{}J'.format(color)] & df['{}9'.format(color)]
            feature_columns.append(new_col)
            
            # Exercise: Add other features here such as the combination of Ace-King-Queen (Dreiblatt).
            # todo

            akq_col = '{}_AKQ'.format(color)
            df[akq_col]  = df['{}A'.format(color)] & df['{}K'.format(color)] & df['{}Q'.format(color)]
            feature_columns.append(akq_col)

        trumpSelector = pickle.load(open("trumpSelector", 'rb'))
        print(trumpSelector)
        trumpResult = trumpSelector.predict(df)
        print(trumpResult)
        helper = {0: 'DIAMONDS', 1: 'HEARTS', 2: 'SPADES', 3:'CLUBS',
                                  4: 'OBE_ABE', 5: 'UNE_UFE', 10: 'PUSH'}
        trumpResultId = (list(helper.keys())[list(helper.values()).index(trumpResult[0])])  # Prints george

        if (obs.forehand == -1 or obs.declared_trump == -1) and trumpResultId == 10:
            card_list = convert_one_hot_encoded_cards_to_int_encoded_list(valid_cards)
            best_score = 0
            best_color = -1
            for color in trump_ints:
                score = calculate_trump_selection_score(card_list, color)
                if best_score < score:
                    best_score = score
                    best_color = color
            return best_color


        return trumpResultId

        # card_list = convert_one_hot_encoded_cards_to_int_encoded_list(valid_cards)
        # best_score = 0
        # best_color = -1
        # for color in trump_ints:
        #     score = calculate_trump_selection_score(card_list, color)
        #     if best_score < score:
        #         best_score = score
        #         best_color = color

        # if best_score < 68 and obs.declared_trump < 0:
        #     return PUSH
        # return best_color

        
        

    def action_play_card(self, obs: GameObservation) -> int:
        """
        Determine the card to play.

        Args:
            obs: the game observation

        Returns:
            the card to play, int encoded as defined in jass.game.const
        """
        valid_cards = self._rule.get_valid_cards_from_obs(obs)
        
        print(valid_cards)
        encoded_cards = convert_one_hot_encoded_cards_to_int_encoded_list(valid_cards)
        if len(encoded_cards) == 4: 
            xIn =  { "points":1, "card1":encoded_cards[0], "card2":encoded_cards[1], "card3":encoded_cards[2], "card4":encoded_cards[3],"first": obs.forehand, "trump":obs.trump}

            print(xIn["points"])
            #points	win	first	card1	card2	card3	card4	trump
            loaded_scaler = pickle.load(open("D:\hslul\DL4G\DL4G-HSLU\sw01\scaler_file", 'rb'))

            #scaler.transform([[4,1,10,4,45,1,1]])
            #'card1','card2','card3','card4', 'points', 'first', 'trump'
            numberInput = [xIn["card1"],xIn["card2"],xIn["card3"],xIn["card4"], xIn["points"], xIn["first"], xIn["trump"] ]
            print(numberInput);
            normalizedInput = loaded_scaler.transform([numberInput])
            loaded_model = pickle.load(open("D:\hslul\DL4G\DL4G-HSLU\sw01\knnpickle_file", 'rb'))
            result = loaded_model.predict(normalizedInput)
            self.hasStrongHand = result > 0.8
        # else:
        #     print("Not start round")
        
        # if self.hasStrongHand:
        #     # use aggresive strategy
        # else:
        #     # supportive strategy
        #     print("oh no my cards kinda stink")

        


        # we use the global random number generator here
        return np.random.choice(np.flatnonzero(valid_cards))

We can use the game simulation to play a game. We will use that to test our implementation, and then use the arena class to play against other agents

In [195]:
rule = RuleSchieber()
game = GameSim(rule=rule)
agent = MyAgent()

np.random.seed(1)
game.init_from_cards(hands=deal_random_hand(), dealer=NORTH)

In [196]:
obs = game.get_observation()

In [197]:
cards = convert_one_hot_encoded_cards_to_str_encoded_list(obs.hand)
print(cards)
trump = agent.action_trump(obs)
assert trump == HEARTS

['DA', 'DK', 'D9', 'D6', 'HA', 'HQ', 'HJ', 'H8', 'H7']
   DA  DK  DQ  DJ  D10  D9  D8  D7  D6  HA  ...  CA  CK  CQ  CJ  C10  C9  C8  \
0   1   1   0   0    0   1   0   0   1   1  ...   0   0   0   0    0   0   0   

   C7  C6  FH  
0   0   0   0  

[1 rows x 37 columns]
   DA  DK  DQ  DJ  D10  D9  D8  D7  D6  HA  ...  CA  CK  CQ  CJ  C10  C9  C8  \
0   1   1   0   0    0   1   0   0   1   1  ...   0   0   0   0    0   0   0   

   C7  C6  FH  
0   0   0   0  

[1 rows x 37 columns]
LogisticRegression(C=0.7, max_iter=1000, multi_class='multinomial')
['PUSH']


In [198]:
# tell the simulation the selected trump
game.action_trump(trump)

In [199]:
# play the game to the end and print the result
while not game.is_done():
    game.action_play_card(agent.action_play_card(game.get_observation()))

print(game.state.points)

[1 1 0 0 0 1 0 0 1 1 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1
[2, 4, 10, 14, 1, 1, 1]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0]
[1 0 0 0 0 1 0 0 1 1 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0

Another possibility to test agents locally is to use the arena. Let us play 100 games against the Random Agent and see if our trump methods makes any difference.


In [200]:
arena = Arena(nr_games_to_play=100)
arena.set_players(MyAgent(), AgentRandomSchieber(), MyAgent(), AgentRandomSchieber())

In [201]:
arena.play_all_games()

[1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
1
[1, 20, 21, 24, 1, 0, 0]
[0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0]
[1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1
[1, 11, 12, 13, 1, 0, 0]
[1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
1
[0, 3, 5, 32, 1, 0, 0]
[0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1
[3, 4, 5, 6, 1, 0, 0]
[0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
1
[11, 12

In [202]:
print(arena.points_team_0.sum(), arena.points_team_1.sum())

8111.0 7589.0


Test Bot:

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
 # import seaborn as sns

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline

from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import KFold

from tqdm.notebook import tqdm
import ipywidgets as widgets
%matplotlib inline

from jass.game.game_util import *
from jass.game.game_sim import GameSim
from jass.game.game_observation import GameObservation
from jass.game.const import *
from jass.game.rule_schieber import RuleSchieber
from jass.agents.agent import Agent
from jass.agents.agent_random_schieber import AgentRandomSchieber
from jass.arena.arena import Arena

read data

In [44]:
df = pd.read_json("F:\jass_game_0001.txt", lines=True)

In [47]:
df.shape
first_column = df.iloc[:, 0]
print(first_column)
dfItem = pd.DataFrame.from_records(first_column)
print(dfItem.tricks)
df.head(n=5)
dfItem.head(n=5)

#print(dfItem)
items = pd.DataFrame.from_records(dfItem);
#df.loc[:, numerical_cols]
print(items.iloc[[1]])

0        {'version': 'V0.2', 'trump': 1, 'dealer': 2, '...
1        {'version': 'V0.2', 'trump': 1, 'dealer': 0, '...
2        {'version': 'V0.2', 'trump': 3, 'dealer': 1, '...
3        {'version': 'V0.2', 'trump': 2, 'dealer': 3, '...
4        {'version': 'V0.2', 'trump': 1, 'dealer': 1, '...
                               ...                        
99995    {'version': 'V0.2', 'trump': 1, 'dealer': 3, '...
99996    {'version': 'V0.2', 'trump': 3, 'dealer': 3, '...
99997    {'version': 'V0.2', 'trump': 3, 'dealer': 2, '...
99998    {'version': 'V0.2', 'trump': 2, 'dealer': 1, '...
99999    {'version': 'V0.2', 'trump': 2, 'dealer': 2, '...
Name: game, Length: 100000, dtype: object
0        [{'cards': ['HA', 'H9', 'H10', 'D10'], 'points...
1        [{'cards': ['H8', 'H9', 'HQ', 'H10'], 'points'...
2        [{'cards': ['CA', 'CJ', 'C7', 'D10'], 'points'...
3        [{'cards': ['S10', 'SQ', 'S9', 'S6'], 'points'...
4        [{'cards': ['HA', 'HJ', 'H6', 'H8'], 'points':...
              

prepare data

In [48]:
tricks = pd.DataFrame.from_records(dfItem.tricks);
#tricks.head()
# df.loc[:, numerical_cols]

values = []

for i in range(tricks.shape[0]): #iterate over rows
    headerInfo = items.iloc[[i]]
    for j in range(tricks.shape[1]): #iterate over columns
        value = tricks.at[i, j] #get cell value
       # print(value["cards"])
       # print(convert_str_encoded_cards_to_int_encoded(value["cards"]))
        value["cards"] = convert_str_encoded_cards_to_int_encoded(value["cards"])

        value["card1"] = value["cards"][0]
        value["card2"] = value["cards"][1]
        value["card3"] = value["cards"][2]
        value["card4"] = value["cards"][3]
        #print(headerInfo.trump.astype(int).array[0])
        value["trump"] = headerInfo.trump.astype(int).array[0]
        values.append(value);
        #print(value, end="\t")
    #print()
#print(tricks[0][0])
print("DONE");

DONE


convert to dataframe

In [49]:
dfValues = pd.DataFrame.from_records(values)
dfValues.head(5)

,cards,points,win,first,card1,card2,card3,card4,trump
0,"[9, 14, 13, 4]",45,0,1,9,14,13,4,1
1,"[30, 27, 34, 31]",23,3,0,30,27,34,31,1
2,"[26, 22, 20, 23]",13,1,3,26,22,20,23,1
3,"[18, 35, 16, 25]",11,3,1,18,35,16,25,1
4,"[19, 21, 24, 33]",6,3,3,19,21,24,33,1


define features

In [50]:
feature_columns = ['card1','card2','card3','card4', 'points', 'first', 'trump']
#dfValues.drop("cards")

X = dfValues[feature_columns].values
print(X)
# We convert it to float so we don't get a conversion warning when normalizing the data
X = X.astype("float") 
y = dfValues.trump.values

[[ 9 14 13 ... 45  1  1]
 [30 27 34 ... 23  0  1]
 [26 22 20 ... 13  3  1]
 ...
 [29 30 33 ... 16  3  2]
 [ 7  5  0 ... 13  0  2]
 [31 28  1 ... 23  2  2]]


test / train data

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X_train:", X_train.shape)
print("y_train", y_train.shape)
print("X_test:", X_test.shape)
print("y_test", y_test.shape)

X_train: (720000, 7)
y_train (720000,)
X_test: (180000, 7)
y_test (180000,)


KNN default: scaler

In [181]:
print("Original", X_train[0])
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

# You could also call fit_transform, which calls fit first and then transforms the data.
# X_train_scaled = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)
print("Mean per feature:", scaler.mean_)
print("Variance per feature", scaler.var_)

Original [-1.37429949 -0.24672624  0.52459337 -1.50360621  0.62781192  1.34179907
 -1.57238793]
Mean per feature: [-2.20299704e-16  1.23181789e-16  2.26424496e-15  4.57124770e-16
  5.10014870e-17  5.19131343e-16  1.37877904e-15]
Variance per feature [1. 1. 1. 1. 1. 1. 1.]


KNN

In [53]:
knr = KNeighborsRegressor(n_neighbors=5)
knr.fit(X_train, y_train)

KNeighborsRegressor()

KNN Prediction

In [54]:
y_pred = knr.predict(X_test)
y_pred_train = knr.predict(X_train)

# analysis
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("MAE:", mae)
print("r2:", r2)

MAE: 0.004966666666666667
r2: 0.9993904476617756


In [72]:
print(X_train)


result = knr.predict(scaler.transform([[4,1,10,4,45,1,1]]))
print(result)
print(result)


[[-1.37429949 -0.24672624  0.52459337 ...  0.62781192  1.34179907
  -1.57238793]
 [-0.41396089 -1.11344842 -0.72812377 ... -0.04981279  1.34179907
   0.20150893]
 [-0.89413019 -1.40235581 -1.69175233 ... -0.5015626   1.34179907
   0.20150893]
 ...
 [-0.02982545 -0.15042378  1.48822193 ... -1.63093712  1.34179907
   0.20150893]
 [ 1.02654701 -0.24672624  0.91004479 ... -0.27568769 -0.44646164
  -1.57238793]
 [-1.18223177 -0.43933117 -1.01721233 ...  0.85368683 -1.34059199
  -0.38979002]]
[1.]
[1.]


In [182]:
import pickle 

# Its important to use binary mode 
knnPickle = open('knnpickle_file', 'wb') 
      
# source, destination 
pickle.dump(knr, knnPickle)  

# close the file
knnPickle.close()

# Its important to use binary mode 
scalerPickle = open('scaler_file', 'wb') 
      
# source, destination 
pickle.dump(scaler, scalerPickle)  

# close the file
knnPickle.close()

In [1]:
import pickle

from sklearn.linear_model import LogisticRegression 

trumpSelector = LogisticRegression(C=0.7, multi_class='multinomial', penalty='l2', max_iter=1000 )
trumpPickle = open('trumpSelector', 'wb') 
      
# source, destination 
pickle.dump(trumpSelector, trumpPickle)  

# close the file
trumpPickle.close()


Now you can continue with a rule based implemenation of the card play. Also look at the flask implementation of the service to see how you can get your agent online.